In [1]:
import os.path
import glob
import pandas as pd
import re

In [2]:
PRE_TEXT = r"""
\begin{table}[H]
\begin{tabular}{lll|ll|ll}
\hline
 & & & \multicolumn{2}{c|}{$\alpha$=0.05} & \multicolumn{2}{c}{$\alpha$=0.1} \\ \hline
\multicolumn{1}{l|}{n} & \multicolumn{1}{l|}{$\beta(t)$} & $\delta$ &  $H_0=c$  &  $H_0=0$  & $H_0=c$ & $H_0=0$ \\ \hline
"""

POST_TEXT = r"""
\end{tabular}
\caption{Rejection Rate(Standard Error) for $5000$ simulations when the number of time points is $<timepoints_value/>$ using RLRT and L2 penalty for non-testing item }
\label{typeI<timepoints_value/>l2gam}
\end{table}
"""

In [3]:
files = glob.glob("/Users/xzhao17/Documents/GitHub/xcj_hypothesis_test_cfd/gam_*.csv")
#files=files[0]
files

['/Users/xzhao17/Documents/GitHub/xcj_hypothesis_test_cfd/gam_90.csv',
 '/Users/xzhao17/Documents/GitHub/xcj_hypothesis_test_cfd/gam_180.csv']

In [4]:
line_part1 = '{p1} & {delta} & {fl_choice}'
p1_1 = '\\multicolumn{{1}}{{l|}}{{\\multirow{{4}}{{*}}{{{num_subjects}}}}}'
p1_2 = '\\multicolumn{{1}}{{l|}}{{}}'
line_part2 = ' & {} & {} & {} & {} \\\\'
print(line_part1.format(p1=300, delta="\multicolumn{1}{l|}{}", fl_choice=2.5) + line_part2.format(0.0626, 0.003426, 0.1118, 0.004456))

300 & \multicolumn{1}{l|}{} & 2.5 & 0.0626 & 0.003426 & 0.1118 & 0.004456 \\


In [7]:
for file in files:
    timepoints = os.path.basename(file)[:-4].split("_")[-1]
    df = pd.read_csv(file)
    df["fl_choice_factor"] = df["fl_choice"].apply(lambda x: {6:0, 200:1, 7:2, 21:3}[x])
    df.sort_values(["num_subjects", "fl_choice_factor"], inplace=True)

    print(PRE_TEXT)
    flc_to_col1_empty = r"\multicolumn{1}{l|}{}"
    flc_to_col2 = {0: r"\multicolumn{1}{l|}{}", 2: r"\multicolumn{1}{l|}{}", 1: r"\multicolumn{1}{c|}{$\delta$}", 3: r"\multicolumn{1}{l|}{$1+\delta$ t}"}
    flc_to_col3 = {0: 0, 1: 2.5, 2: 5, 3: 0}
    for num_subj in sorted(df["num_subjects"].unique()):
        for flc in sorted(df["fl_choice_factor"].unique()):
            p1 = p1_1.format(num_subjects=num_subj) if flc == 0 else p1_2.format(num_subjects=num_subj)
            recf = df[(df["fl_choice_factor"] == flc) & (df["num_subjects"] == num_subj) & (df["test_type"] == "Functional")]
            reci = df[(df["fl_choice_factor"] == flc) & (df["num_subjects"] == num_subj) & (df["test_type"] == "Inclusion")]
            f05 = "{:.04f} ({:.04f})".format(recf.iloc[0]["power"], recf.iloc[0]["se"])
            i05 = "{:.04f} ({:.04f})".format(reci.iloc[0]["power"], reci.iloc[0]["se"]) if reci.shape[0] > 0 else ""
            f10 = "{:.04f} ({:.04f})".format(recf.iloc[0]["power_01"], recf.iloc[0]["se01"])
            i10 = "{:.04f} ({:.04f})".format(reci.iloc[0]["power_01"], reci.iloc[0]["se01"]) if reci.shape[0] > 0 else ""
            print(line_part1.format(p1=p1, delta=flc_to_col2[flc], fl_choice=flc_to_col3[flc]) + line_part2.format(f05, i05, f10, i10))
        print(r"\hline")
    print(re.sub("<timepoints_value/>", f"{timepoints}", POST_TEXT))


\begin{table}[H]
\begin{tabular}{lll|ll|ll}
\hline
 & & & \multicolumn{2}{c|}{$\alpha$=0.05} & \multicolumn{2}{c}{$\alpha$=0.1} \\ \hline
\multicolumn{1}{l|}{n} & \multicolumn{1}{l|}{$\beta(t)$} & $\delta$ &  $H_0=c$  &  $H_0=0$  & $H_0=c$ & $H_0=0$ \\ \hline

\multicolumn{1}{l|}{\multirow{4}{*}{100}} & \multicolumn{1}{l|}{} & 0 & 0.0566 (0.0033) & 0.0530 (0.0032) & 0.1160 (0.0045) & 0.1088 (0.0044) \\
\multicolumn{1}{l|}{} & \multicolumn{1}{c|}{$\delta$} & 2.5 & 0.0582 (0.0033) &  & 0.1112 (0.0044) &  \\
\multicolumn{1}{l|}{} & \multicolumn{1}{l|}{} & 5 & 0.0616 (0.0034) &  & 0.1156 (0.0045) &  \\
\multicolumn{1}{l|}{} & \multicolumn{1}{l|}{$1+\delta$ t} & 0 & 0.0580 (0.0033) &  & 0.1126 (0.0045) &  \\
\hline
\multicolumn{1}{l|}{\multirow{4}{*}{300}} & \multicolumn{1}{l|}{} & 0 & 0.0572 (0.0033) & 0.0562 (0.0033) & 0.1100 (0.0044) & 0.1028 (0.0043) \\
\multicolumn{1}{l|}{} & \multicolumn{1}{c|}{$\delta$} & 2.5 & 0.0574 (0.0033) &  & 0.1074 (0.0044) &  \\
\multicolumn{1}{l|}{} & \mult